Importing Libraries


In [ ]:
!pip install torch trl
!pip install -U accelerate peft safetensors transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 72.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


In [ ]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.8 MB/s eta 0:00:00


Importing to test and train dataframes


In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import pandas as pd

file_path = "train-test-data.xlsx"

train_df = pd.read_excel(file_path, sheet_name="train")
test_df  = pd.read_excel(file_path, sheet_name="test")

print("Train dataset sample:")
display(train_df.head())

print("Test dataset sample:")
display(test_df.head())

Train dataset sample:


,case_id,input_finding,output_disease
0,1,The liver surface is regular with no apparent ...,Liver lesion
1,2,"The liver and spleen are in normal position, s...","Kidney stone, Pancreatitis"
2,3,"The liver surface is smooth, and the size and ...","Liver cyst, Renal cyst, Splenic cyst"
3,4,"Abnormal liver morphology, with arc-shaped low...","Liver lesion, Liver cyst, Gallstone, Ascites, ..."
4,5,"Two round low-density lesions, approximately 5...","Liver lesion, Splenomegaly, Renal mass, Colore..."


Test dataset sample:


,case_id,input_finding,output_disease
0,1,The liver is normal in size and shape with hom...,"Renal cyst, Adrenal hyperplasia, Adrenal calci..."
1,2,"The liver is normal in position, size, and sha...","Liver lesion, Atherosclerosis"
2,3,"The liver surface is smooth, with coordinated ...","Liver cyst, Pancreas calcifications, Accessory..."
3,4,The liver surface is smooth with coordinated s...,"Cholecystitis, Renal cyst, Ascites"
4,5,"The surface of the liver is smooth, and the si...","Liver calcifications, Gallstone"


Importing the Qwen3-8B model in 4-bit

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Filtering all the possible outputs of the diseases since it is a multiclass output

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
df = pd.read_excel("train-test-data.xlsx")
label_lists = [list(dict.fromkeys(lbl.split(", "))) for lbl in df['output_disease']]

mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(label_lists)

classes = mlb.classes_
print("All disease classes:", classes)
print("Multi-label encoded shape:", labels_encoded.shape)

All disease classes: ['Abdominal abscess' 'Abdominal aortic aneurysm' 'Abdominal lipiditis'
 'Abdominal mass' 'Absent kidney' 'Accessory spleen'
 'Adrenal calcification' 'Adrenal hyperplasia' 'Adrenal mass'
 'Adrenal metastasis' 'Adrenal nodule' 'Annular pancreas' 'Appendicitis'
 'Arteriosclerosis' 'Ascites' 'Atherosclerosis' 'Basal cell carcinoma'
 'Bile duct cancer' 'Bile duct carcinoma' 'Bile duct dilatation'
 'Bile duct stone' 'Cecal epiploic appendagitis' 'Cholecystitis'
 'Cirrhosis' 'Colitis' 'Colorectal cancer' 'Diverticulitis' 'Enteritis'
 'Enteritis possible' 'Epiploic appendagitis' 'Fatty liver'
 'Gallbladder mass' 'Gallstone' 'Gastric cancer' 'Gastric lymphoma'
 'Gastrointestinal bleed' 'Gastrointestinal perforation' 'Hepatitis'
 'Hernia' 'Hydronephrosis' 'Inflammatory bowel disease'
 'Intestinal obstruction' 'Kidney infarction' 'Kidney stone'
 'Liver abscess' 'Liver calcification' 'Liver cyst' 'Liver lesion'
 'Lymphadenopathy' 'Neuroblastoma' 'No acute abnormality'
 'Omenta

Clasification based on input findings

In [ ]:
labels = classes
def classify_finding(input_finding: str):
    system_msg = (
        """
You are a radiology assistant. Extract all diseases mentioned in the report.
Use only the known disease classes listed below. Do not invent any new diseases.
Return your answer as a comma-separated list.

Known disease classes: {labels}

Report: "{input_fiding}"
Output:

Example 1:
Report: "The liver is normal in size and shape with homogeneous density. A patchy low-density lesion is seen around the liver fissure. The intrahepatic duct system is not obviously dilated, and the course is normal. The gallbladder is not enlarged, with no obvious thickening of the wall, and no clearly abnormal density foci are seen inside. The spleen is normal in size and shape with homogeneous density, and some punctate low-density foci are seen inside. The pancreas is normal in size and shape with homogeneous density, and no clearly abnormal density foci are seen inside. The main pancreatic duct is not obviously dilated, and the peripancreatic fat space is clear. Both kidneys are normal in size and shape with homogeneous density. A round low-density lesion is seen in the right kidney with a diameter of about 16mm. The left adrenal gland is thickened, and a punctate high-density lesion is seen in the right adrenal gland. The renal pelvis-calyx system is not obviously dilated. The perirenal fat space is clear, and no clearly abnormal density foci are seen. No enlarged lymph nodes are seen in the retroperitoneum."
Output: ["Renal cyst", "Adrenal hyperplasia", "Adrenal calcification"]

Example 2:
Report: "Lungs are clear. Heart normal."
Output: []

Now analyze this report:

Report: "{input_finding}"
Output:
"""
    )
    user_msg = f"input_finding: {input_finding}\noutput_disease:"
    prompt = (
        "<|im_start|>system\n" + system_msg + "<|im_end|>\n"
        "<|im_start|>user\n" + user_msg + "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=4,
            do_sample=False,
            num_beams=1
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = text.split("output_disease:")[-1].strip().split()[0]
    return pred


Converting the dataframe data into Qwen acceptible format by assigning the user, system and assistant roles

In [ ]:
import pandas as pd
from datasets import Dataset


def to_conv(example):
    system_msg = (
        "You are a medical text classifier. "
        "Reply with exactly one disease label from the allowed list."
    )
    user_msg = f"input_finding: {example['input_finding']}\noutput_disease:"
    assistant_msg = example["output_disease"]
    return {
        "conversations": [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg},
        ]
    }

dataset = Dataset.from_pandas(train_df)
dataset = dataset.map(to_conv)


Map:   0%|          | 0/1236 [00:00<?, ? examples/s]

Formatting as per the function prompts

In [ ]:
def formatting_prompts_func(examples):
    texts = []
    for conv in examples["conversations"]:
        text = tokenizer.apply_chat_template(
            conv,
            tokenize=False,
            add_generation_prompt=False,
        )
        texts.append(text)
    return {"text": texts}

dataset_text = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names,
)


Map:   0%|          | 0/1236 [00:00<?, ? examples/s]

Usig LoRA to identify model, traininable params and total params

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,833,856 || all params: 8,194,569,216 || trainable%: 0.0468


Fine tuning the model using LoRA

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="qwen3-8b-med-multiclass",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    save_strategy="epoch",
    max_steps=-1,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_text,
    args=training_args,
    processing_class=tokenizer,
)

trainer.train()


Adding EOS to train dataset:   0%|          | 0/1236 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1236 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1236 [00:00<?, ? examples/s]

Step,Training Loss
10,2.270200
20,2.073600
30,1.846800
40,1.761800
50,1.589900
60,1.481500
70,1.383600
80,1.320500
90,1.274600
100,1.259000


TrainOutput(global_step=310, training_loss=1.2972305144033125, metrics={'train_runtime': 6836.711, 'train_samples_per_second': 0.362, 'train_steps_per_second': 0.045, 'total_flos': 2.543400102985728e+16, 'train_loss': 1.2972305144033125, 'epoch': 2.0})

Saving model after fine tuning

In [ ]:
save_directory = "./qwen3-8b-med-multiclass-finetuned"
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)


('./qwen3-8b-med-multiclass-finetuned/tokenizer_config.json',
 './qwen3-8b-med-multiclass-finetuned/special_tokens_map.json',
 './qwen3-8b-med-multiclass-finetuned/chat_template.jinja',
 './qwen3-8b-med-multiclass-finetuned/vocab.json',
 './qwen3-8b-med-multiclass-finetuned/merges.txt',
 './qwen3-8b-med-multiclass-finetuned/added_tokens.json',
 './qwen3-8b-med-multiclass-finetuned/tokenizer.json')

In [ ]:
!zip -r qwen3_finetuned.zip ./qwen3-8b-med-multiclass-finetuned


  adding: qwen3-8b-med-multiclass-finetuned/ (stored 0%)
  adding: qwen3-8b-med-multiclass-finetuned/README.md (deflated 65%)
  adding: qwen3-8b-med-multiclass-finetuned/training_args.bin (deflated 53%)
  adding: qwen3-8b-med-multiclass-finetuned/adapter_model.safetensors (deflated 8%)
  adding: qwen3-8b-med-multiclass-finetuned/special_tokens_map.json (deflated 69%)
  adding: qwen3-8b-med-multiclass-finetuned/adapter_config.json (deflated 57%)
  adding: qwen3-8b-med-multiclass-finetuned/chat_template.jinja (deflated 76%)
  adding: qwen3-8b-med-multiclass-finetuned/merges.txt (deflated 57%)
  adding: qwen3-8b-med-multiclass-finetuned/tokenizer.json (deflated 81%)
  adding: qwen3-8b-med-multiclass-finetuned/tokenizer_config.json (deflated 90%)
  adding: qwen3-8b-med-multiclass-finetuned/added_tokens.json (deflated 68%)
  adding: qwen3-8b-med-multiclass-finetuned/vocab.json (deflated 61%)


In [ ]:
from google.colab import files
files.download("qwen3_finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import zipfile

# Path to the zip file
zip_path = '/content/qwen3_finetuned 8b.zip'

# Destination directory
extract_path = '/content/qwen3-8b-med-multiclass/'

# Extracting all files
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_path)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig
from accelerate import disk_offload
import torch
import os
import pandas as pd

# Paths and settings
model_name = "Qwen/Qwen3-8B"
offload_dir = "./offload_dir"
adapter_path = '/content/qwen3-8b-med-multiclass/qwen3-8b-med-multiclass-finetuned'
os.makedirs(offload_dir, exist_ok=True)

# Load base model to CPU with low memory usage
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=None,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16
)

# Explicitly offload weights to disk to handle large model
disk_offload(base_model, offload_dir)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load PEFT LoRA config and prepare model
peft_config = LoraConfig.from_pretrained(adapter_path)
model = get_peft_model(base_model, peft_config)

model.eval()
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

# Your known disease classes list
labels = classes  # Define your list of diseases here

def classify_finding(input_finding: str):
    system_msg = f"""
You are a radiology assistant. Extract all diseases mentioned in the report.
Use only the known disease classes listed below. Do not invent any new diseases.
Return your answer as a comma-separated list.

Known disease classes: {labels}

Report: "{input_finding}"
Output:
"""
    user_msg = f"input_finding: {input_finding}\noutput_disease:"
    prompt = (
        "<|im_start|>system\n" + system_msg + "<|im_end|>\n"
        "<|im_start|>user\n" + user_msg + "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
            num_beams=1,
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = text.split("output_disease:")[-1].strip().split()[0]
    return pred

# Apply predictions to test dataframe
test_df['predicted_disease'] = test_df['input_finding'].apply(classify_finding)

# Calculate accuracy if true labels are in test_df
accuracy = (test_df['predicted_disease'] == test_df['output_disease']).mean()
print(f"Test Accuracy: {accuracy:.4f}")

# Save predictions to file
test_df.to_csv("test_predictions.csv", index=False)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
!ls /content/qwen3-8b-med-multiclass/qwen3-8b-med-multiclass-finetuned

adapter_config.json	   merges.txt		    tokenizer.json
adapter_model.safetensors  README.md		    training_args.bin
added_tokens.json	   special_tokens_map.json  vocab.json
chat_template.jinja	   tokenizer_config.json


In [ ]:
import os
import shutil

offload_dir = "./offload_dir"

# Check if directory exists and is writable
if os.path.exists(offload_dir):
    print(f"{offload_dir} exists")
    if os.access(offload_dir, os.W_OK):
        print(f"{offload_dir} is writable")
    else:
        print(f"{offload_dir} is NOT writable")
else:
    print(f"{offload_dir} does NOT exist")

# Check available disk space on the partition containing offload_dir
total, used, free = shutil.disk_usage(offload_dir)
print(f"Disk space on the partition of {offload_dir}:")
print(f"Total: {total // (2**30)} GB")
print(f"Used: {used // (2**30)} GB")
print(f"Free: {free // (2**30)} GB")


./offload_dir exists
./offload_dir is writable
Disk space on the partition of ./offload_dir:
Total: 112 GB
Used: 53 GB
Free: 58 GB
